# Download biological occurrences data from EMODnet Biology

> An existence of a particular organism or defined group of organisms considered to be taxonomically homogeneous, at a particular place at a particular time

https://dwc.tdwg.org/terms/#occurrence

The EMODnet Biology occurrences data area available via [Web Feature services (WFS)](https://www.ogc.org/publications/standard/wfs/). The most user-friendly way of querying the EMODnet Biology WFS service is using the emodnet.wfs R package. This package allows interrogation of and access to [EMODnet's, European Marine Observation and Data Network, geographic vector data](https://emodnet.ec.europa.eu/en/emodnet-web-service-documentation#inline-nav-3) in R through the [EMODnet Web Feature Services](https://emodnet.ec.europa.eu/en/data-0). [Web Feature services (WFS)](https://www.ogc.org/publications/standard/wfs/) represent a change in the way geographic information is created, modified and exchanged on the Internet and offer direct fine-grained access to geographic information at the feature and feature property level. 
Features are representation of geographic entities, such as a coastlines, marine protected areas, offshore platforms, or fishing areas. In WFS, features have geometry (spatial information) and attributes (descriptive data).
emodnet.wfs aims at offering an user-friendly interface to this rich data.

## Installation

We will use the [emodnet.wfs](https://emodnet.github.io/emodnet.wfs/) R package.

```r
install.packages("pak")
pak::pkg_install("EMODnet/emodnet.wfs")

```

For illustrative purposes, we will use `dplyr` and `stringr` to manipulate dataframes and strings respectively. Install with the code below:

```r
install.packages("dplyr")
install.packages("stringr")

```

## Finding the EMODnet Biology occurrences layers

The package `emodnet.wfs` allows to access all WFS services of all EMODnet thematic lots. There is a dataframe included that shows the different services available.


EMODnet Biology shares infrastructure with EurOBIS - meaning the EMODnet-Biology occurrences data come from the EurOBIS database. EurOBIS is the European node of OBIS.
The easiest way to access EMODnet occurrences programatically is using the `eurobis` R package.

You can filter the data on several parameters. E.g. you can retrieve one single dataset by providing the Integrated Marine Information System ([IMIS](https://www.vliz.be/en/integrated-marine-information-system)) unique identifier for datasets [dasid](https://www.vliz.be/imis?page=webservices).

In [1]:
library(emodnet.wfs)
emodnet_wfs()

ERROR: Error in library(emodnet.wfs): there is no package called ‘emodnet.wfs’


In [ ]:

wfs_bio <- emodnet_init_wfs_client("biology_occurrence_data")

bio_info <- emodnet_get_wfs_info(wfs_bio)

head(bio_info)

bio_info |> 
  select(layer_name, title, abstract) |>
  filter(str_detect(layer_name, "eurobis-obisenv"))

layer_attributes_get_names(wfs_bio, layer = "eurobis-obisenv_basic")
# layer_attribute_descriptions(wfs_bio, layer = "eurobis-obisenv_full")
# layer_attribute_descriptions(wfs_bio, layer = "eurobis-obisenv")

bio_info |> 
  select(layer_name, title, abstract) |>
  filter(layer_name == "eurobis-obisenv_basic") |> 
  pull(abstract)

bio_info |> 
  select(layer_name, title, abstract) |>
  filter(layer_name == "eurobis-obisenv_full") |> 
  pull(abstract)

bio_info |> 
  select(layer_name, title, abstract) |>
  filter(layer_name == "eurobis-obisenv") |> 
  pull(abstract)

#### Taxon
Use a scientific name (e.g. the sea turtle _Caretta caretta_) or a [WoRMS AphiaID](https://www.marinespecies.org/about.php#what_is_aphia) (e.g. [137205](https://www.marinespecies.org/aphia.php?p=taxdetails&id=137205)).


In [16]:
# Download a single dataset
eurobis_occurrences_basic(aphiaid = 137205) |> head()

✔ Downloading layer: EMODnet EurOBIS Basic Occurrence Data

ℹ The Basic Occurrence Data download provides you data for the following 8 essential terms: datasetid, datecollected, decimallongitude, decimallatitude, coordinateuncertaintyinmeters, scientificname, aphiaid, scientificnameaccepted. For more information, please consult: https://www.emodnet-biology.eu/emodnet-data-format.



#### Traits
Get all occurrences that are benthos. Or zooplankton. Or both. Powered by [WoRMS](https://www.marinespecies.org/).

In [ ]:
eurobis_occurrences_basic(functional_group = "phytoplankton") |> head()

#### Time
Just give start and end dates.

In [ ]:
eurobis_occurrences_basic(start_date = "2010-01-01", end_date = "2010-06-31") |> head()

#### Geographically
Query on more than 300 records from the [Marine Regions Gazetteer](https://marineregions.org/gazetteer.php) by giving the [MRGID](https://marineregions.org/mrgid.php). Or just pass the area of your interest as a polygon written in as [Well Known Text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry).

In [ ]:
selected_area <- eurobis_map_draw()
selected_area

# eurobis_occurrences_basic(geometry = selected_area, functional_group = "angiosperms")

Loading required namespace: leaflet.extras

Loading required namespace: miniUI

Loading required package: shiny

Warning message:
“package ‘shiny’ was built under R version 4.4.2”

Listening on http://127.0.0.1:3414



#### Other important classifications
As [IUCN Red List](https://www.iucnredlist.org/en), [MSDF Indicators](https://msfd.eu/knowseas/guidelines/3-INDICATORS-Guideline.pdf) or [Habitats Directive](https://ec.europa.eu/environment/nature/conservation/species/habitats_dir_en.htm) and [CITES](https://cites.org/eng/app/index.php) Annexes.

### The data format

There are [three data formats](https://emodnet.ec.europa.eu/en/biology#biology-data-and-products-format) presented by EMODnet Biology and available through `eurobis`

#### Basic Occurrence Data

The **Basic Occurrence Data** provides you with all data required to perform temporal spatial analysis for the different taxa. It indicates which taxa was found (`scientificname` and `aphiaid`), when (`datecollected`) and where (`decimallongitude` and `decimallatitude` in WGS84 - EPGS:4326), along with a dataset identifier (`datasetid`).

In [ ]:
# Download only basic occurrences
eurobis_occurrences_basic(dasid = 8045) |> head()

#### Full Occurrence Data

The **Full Occurrence Data** provides additional information which may help interpret the basic data. This format offers all the data from the Basic Occurrences plus additiontal information which may help interpret the basic data such as information on the institute collecting the data, the methodology, the exact time and location (and uncertainty),...

In [ ]:
# Download all occurrences information
eurobis_occurrences_full(dasid = 8045) |> head()

#### Full Occurrence Data and Parameters

The **Full Occurrence Data and Parameters** provides you with all measurement or facts associated to the occurrence or the sample. 

In [ ]:
# Download all occurrences and all measurements or facts
eurobis_occurrences_full_and_parameters(dasid = 8045) |> head()

You can find more information about the terms returned by each download type in the [EMODnet Biology documentation](https://emodnet.ec.europa.eu/en/biology#biology-data-and-products-format). 